In [2]:
import torch
from utils import convert_mesh_container_to_torch_mesh
from dataloaders.mesh_container import MeshContainer

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("No GPU available, falling back to CPU.")


No GPU available, falling back to CPU.


In [ ]:
import os

def load_mesh_by_index(folder, index):
    # Get all files in the folder
    files = sorted(os.listdir(folder))
    
    # Filter only the files you want (e.g., by extension)
    valid_extensions = {".obj", ".off"}
    files = [f for f in files if os.path.splitext(f)[1].lower() in valid_extensions]
    
    # Check if the index is valid
    if index < 0 or index >= len(files):
        raise IndexError(f"Index out of range. Valid range: 0-{len(files) - 1}")
    
    # Get the corresponding file
    file_to_load = files[index]
    
    # Construct the full file path
    file_path = os.path.join(folder, file_to_load)
    print(f"Loading file: {file_path}")
    
    # Load the mesh
    mesh = MeshContainer().load_from_file(file_path)
    return mesh


In [ ]:
from diff3f import batch_render

hw = 128
H = hw
W = hw
num_views = 10
use_normal_map = True
# load mesh
folder = "meshes"
index = 5

try:
    source_mesh = load_mesh_by_index(folder, index)
except IndexError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

mesh = convert_mesh_container_to_torch_mesh(source_mesh, device=device, is_tosca=False)

batched_renderings, normal_batched_renderings, camera, depth = batch_render(
    device=device, 
    mesh=mesh, 
    num_views=num_views, 
    H=H, 
    W=W, 
    use_normal_map=use_normal_map, 
    fixed_angle= {'type': 'elevation', 'value': 0}
)

batched_renderings_elev, normal_batched_renderings_elev, camera_elev, depth_elev = batch_render(
    device=device, 
    mesh=mesh, 
    num_views=num_views, 
    H=H, 
    W=W, 
    use_normal_map=use_normal_map, 
    fixed_angle= {'type': 'azimuth', 'value': 0}
)

In [4]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_rotation_sequence(renderings, num_views, save_video=False, display_frames=False, output_path="rotation_video.mp4"):
    renderings = renderings.cpu().numpy()
    
    if save_video:
        import cv2
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 30.0, (W, H))
    
    # Visualize each frame
    for i in range(num_views):
        view = renderings[i]
        
        if save_video:
            # Convert to BGR for OpenCV
            frame = (view * 255).astype(np.uint8)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            out.write(frame)
        
        # Display every 10th frame to check progression
        if display_frames:
            plt.figure(figsize=(8, 8))
            plt.imshow(view)
            plt.title(f"Elevation: {(i/num_views)*360:.1f}°")
            plt.axis("off")
            plt.show()
            plt.close()

    if save_video:
        out.release()
        print(f"Video saved to {output_path}")

In [5]:
import subprocess
import os

def play_video_mac(video_path):
    # Ensure the file exists
    if not os.path.exists(video_path):
        print(f"Error: File not found at {video_path}")
        return
    
    # Open video with the default application
    try:
        subprocess.run(["open", video_path], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [ ]:
combined_renderings = torch.cat([batched_renderings, batched_renderings_elev], axis=0)

# visualize_rotation_sequence(batched_renderings, num_views, save_video=True)
# visualize_rotation_sequence(batched_renderings_elev, num_views, save_video=True)
visualize_rotation_sequence(combined_renderings, num_views*2, save_video=True, display_frames=False, output_path="rotation_video.mp4")



play_video_mac("rotation_video.mp4")

In [3]:
from mesh_video_generator import MeshVideoGenerator

if __name__ == "__main__":
    generator = MeshVideoGenerator(
        output_dir="mesh_videos",
        hw=128,
        num_views=2,
        use_normal_map=True,
        device=device
    )
    
    # Process all meshes in a folder
    generator.process_folder("meshes", display_frames=False)

Found 7 mesh files to process

Processing mesh 1/7
Loading file: meshes/camel.obj
Starting batch_render with num_views=2, H=128, W=128
Fixed angle settings: {'type': 'elevation', 'value': 0}
Rendering completed successfully
Starting batch_render with num_views=2, H=128, W=128
Fixed angle settings: {'type': 'azimuth', 'value': 0}
Rendering completed successfully
Video saved to mesh_videos/camel_combined.mp4

Processing mesh 2/7
Loading file: meshes/cat.off
Starting batch_render with num_views=2, H=128, W=128
Fixed angle settings: {'type': 'elevation', 'value': 0}
Rendering completed successfully
Starting batch_render with num_views=2, H=128, W=128
Fixed angle settings: {'type': 'azimuth', 'value': 0}
Rendering completed successfully
Video saved to mesh_videos/cat_combined.mp4

Processing mesh 3/7
Loading file: meshes/cow.obj
Starting batch_render with num_views=2, H=128, W=128
Fixed angle settings: {'type': 'elevation', 'value': 0}
Rendering completed successfully
Starting batch_render 